In [18]:
import pandas as pd
import numpy as np

from agg import prep_demand_features
from demand_features import X_cols

pd.options.display.max_rows = 150
pd.options.display.max_columns = 250
pd.options.display.max_colwidth = None

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import r2_score

from xgboost import XGBRegressor

In [2]:
h1_stats = pd.read_pickle("../data/h1_stats.pick")
h1_stats["WeekNum"] = h1_stats["WeekNum"].astype(int)

In [3]:
h1_stats["WeekNum"] = h1_stats.StayDate.dt.isocalendar().week

In [4]:
h1_stats[["StayDate", "WeekNum"]].sample(10)

,StayDate,WeekNum
5302,2017-02-03,5
10746,2017-07-27,30
9691,2017-06-25,25
4133,2016-12-12,50
4587,2017-01-01,52
10389,2017-07-11,28
2730,2016-11-03,44
4771,2016-12-30,52
4422,2016-12-22,51
1178,2016-10-01,39


In [5]:
h1_stats.describe()

,RoomsOTB,RevOTB,CxlForecast,TRN_RoomsOTB,TRN_RevOTB,TRN_CxlForecast,DaysUntilArrival,ADR_OTB,SellingPrice,RemSupply,Mon,Sat,Sun,Thu,Tue,Wed,NONTRN_RoomsOTB,NONTRN_RevOTB,NONTRN_ADR_OTB,NONTRN_CxlForecast,ACTUAL_RoomsSold,ACTUAL_ADR,ACTUAL_RoomRev,ACTUAL_NumCancels,WeekNum,TRN_ADR_OTB,TM30_RoomsPickup,TM30_RevPickup,TM30_ADR_Pickup,TM30_TRN_RoomsPickup,TM30_TRN_RevPickup,TM30_TRN_ADR_Pickup,TM30_NONTRN_RoomsPickup,TM30_NONTRN_RevPickup,TM30_NONTRN_ADR_Pickup,TM15_RoomsPickup,TM15_RevPickup,TM15_ADR_Pickup,TM15_TRN_RoomsPickup,TM15_TRN_RevPickup,TM15_TRN_ADR_Pickup,TM15_NONTRN_RoomsPickup,TM15_NONTRN_RevPickup,TM15_NONTRN_ADR_Pickup,TM05_RoomsPickup,TM05_RevPickup,TM05_ADR_Pickup,TM05_TRN_RoomsPickup,TM05_TRN_RevPickup,TM05_TRN_ADR_Pickup,TM05_NONTRN_RoomsPickup,TM05_NONTRN_RevPickup,TM05_NONTRN_ADR_Pickup,RoomsGapToLYA,RevGapToLYA,ADR_GapToLYA,TRN_RoomsGapToLYA,TRN_RevGapToLYA,TRN_ADR_GapToLYA,NONTRN_RoomsGapToLYA,NONTRN_RevGapToLYA,NONTRN_ADR_GapToLYA,DaysUntilArrival_STLY,RoomsOTB_Pace,ADR_OTB_Pace,RevOTB_Pace,CxlForecast_Pace,RemSupply_Pace,SellingPrice_Pace,TRN_RoomsOTB_Pace,TRN_ADR_OTB_Pace,TRN_RevOTB_Pace,TRN_CxlForecast_Pace,NONTRN_RoomsOTB_Pace,NONTRN_ADR_OTB_Pace,NONTRN_RevOTB_Pace,NONTRN_CxlForecast_Pace,TM30_RoomsPickup_Pace,TM30_ADR_Pickup_Pace,TM30_RevPickup_Pace,TM30_TRN_RoomsPickup_Pace,TM30_TRN_ADR_Pickup_Pace,TM30_TRN_RevPickup_Pace,TM30_NONTRN_RoomsPickup_Pace,TM30_NONTRN_ADR_Pickup_Pace,TM30_NONTRN_RevPickup_Pace,TM15_RoomsPickup_Pace,TM15_ADR_Pickup_Pace,TM15_RevPickup_Pace,TM15_TRN_RoomsPickup_Pace,TM15_TRN_ADR_Pickup_Pace,TM15_TRN_RevPickup_Pace,TM15_NONTRN_RoomsPickup_Pace,TM15_NONTRN_ADR_Pickup_Pace,TM15_NONTRN_RevPickup_Pace,TM05_RoomsPickup_Pace,TM05_ADR_Pickup_Pace,TM05_RevPickup_Pace,TM05_TRN_RoomsPickup_Pace,TM05_TRN_ADR_Pickup_Pace,TM05_TRN_RevPickup_Pace,TM05_NONTRN_RoomsPickup_Pace,TM05_NONTRN_ADR_Pickup_Pace,TM05_NONTRN_RevPickup_Pace
count,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.00000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.00000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.00000,12208.000000,12208.000000
mean,139.693562,14401.099538,18.410550,89.930374,10275.610964,16.057585,15.276540,94.521588,101.740257,65.716989,0.142857,0.142038,0.142448,0.144004,0.143267,0.143676,49.763188,4125.488574,82.780837,2.352965,156.167595,94.948845,15719.184259,77.098050,26.820691,100.618447,9.912516,939.082892,0.918850,7.298984,703.86121,0.953703,2.613532,235.221682,1.081473,0.781045,17.742694,-0.264474,0.757700,36.606579,-0.334466,0.023345,-18.863885,-0.359994,-7.672592,-606.246786,-0.258184,-5.445773,-453.559906,-0.251895,-2.226818,-152.686880,-0.395004,4.550704,-1294.865487,-11.824486,10.010321,-469.331316,-14.292563,-5.459617,-825.534172,2.468077,15.276540,12.729112,12.460346,2544.981504,3.797919,-8.931193,15.078701,5.844856,14.970347,1603.077616,4.5

In [6]:
# mask = (h1_stats.Thu == 0) & (h1_stats.Wed == 0) & (h1_stats.Tue == 0) & (h1_stats.Mon == 0) & (h1_stats.Sun == 0)
# h1_stats["WE"] = mask


In [7]:
mask = (h1_stats["StayDate"] < '2017-08-01')
test_mask = (h1_stats['AsOfDate'] == '2017-08-01')
h1_train = h1_stats.loc[mask].copy()
h1_test = h1_stats.loc[test_mask].copy()

X1_train = h1_train[X_cols].copy()
X1_test = h1_test[X_cols].copy()
y1_train = h1_train['ACTUAL_RoomsSold'].copy()
y1_test = h1_test['ACTUAL_RoomsSold'].copy()

# not sure why I need to do this again (it's done in agg.py), but here we are. 
X1_train["WeekNum"] = X1_train.WeekNum.astype('float')
X1_test["WeekNum"] = X1_test.WeekNum.astype('float')

In [61]:
[c for c in h1_stats.columns]

['id',
 'DOW',
 'RoomsOTB',
 'RevOTB',
 'CxlForecast',
 'TRN_RoomsOTB',
 'TRN_RevOTB',
 'TRN_CxlForecast',
 'StayDate',
 'WE',
 'WD',
 'STLY_StayDate',
 'DaysUntilArrival',
 'ADR_OTB',
 'SellingPrice',
 'AsOfDate',
 'STLY_AsOfDate',
 'RemSupply',
 'Mon',
 'Sat',
 'Sun',
 'Thu',
 'Tue',
 'Wed',
 'NONTRN_RoomsOTB',
 'NONTRN_RevOTB',
 'NONTRN_ADR_OTB',
 'NONTRN_CxlForecast',
 'ACTUAL_RoomsSold',
 'ACTUAL_ADR',
 'ACTUAL_RoomRev',
 'ACTUAL_NumCancels',
 'WeekNum',
 'TRN_ADR_OTB',
 'TM30_RoomsPickup',
 'TM30_RevPickup',
 'TM30_ADR_Pickup',
 'TM30_TRN_RoomsPickup',
 'TM30_TRN_RevPickup',
 'TM30_TRN_ADR_Pickup',
 'TM30_NONTRN_RoomsPickup',
 'TM30_NONTRN_RevPickup',
 'TM30_NONTRN_ADR_Pickup',
 'TM15_RoomsPickup',
 'TM15_RevPickup',
 'TM15_ADR_Pickup',
 'TM15_TRN_RoomsPickup',
 'TM15_TRN_RevPickup',
 'TM15_TRN_ADR_Pickup',
 'TM15_NONTRN_RoomsPickup',
 'TM15_NONTRN_RevPickup',
 'TM15_NONTRN_ADR_Pickup',
 'TM05_RoomsPickup',
 'TM05_RevPickup',
 'TM05_ADR_Pickup',
 'TM05_TRN_RoomsPickup',
 'TM05_TR

In [ ]:
X1_train.shape

In [8]:
X1_train.head()

,WeekNum,RoomsOTB,RevOTB,CxlForecast,TRN_RoomsOTB,WE,DaysUntilArrival,ADR_OTB,SellingPrice,RemSupply,Mon,Sat,Sun,Thu,Tue,Wed,NONTRN_RoomsOTB,TM30_RoomsPickup,TM30_TRN_RoomsPickup,TM30_NONTRN_RoomsPickup,TM15_RoomsPickup,TM15_TRN_RoomsPickup,TM15_NONTRN_RoomsPickup,TM05_RoomsPickup,TM05_TRN_RoomsPickup,TM05_NONTRN_RoomsPickup,RoomsGapToLYA,RevGapToLYA,TRN_RoomsGapToLYA,TRN_RevGapToLYA,NONTRN_RoomsGapToLYA,RoomsOTB_Pace,RemSupply_Pace,SellingPrice_Pace,TRN_RoomsOTB_Pace,NONTRN_RoomsOTB_Pace,TM30_RoomsPickup_Pace,TM30_TRN_RoomsPickup_Pace,TM30_NONTRN_RoomsPickup_Pace,TM15_RoomsPickup_Pace,TM15_TRN_RoomsPickup_Pace,TM15_NONTRN_RoomsPickup_Pace,TM05_RoomsPickup_Pace,TM05_TRN_RoomsPickup_Pace,TM05_NONTRN_RoomsPickup_Pace
0,30.0,170.0,28570.36,22.0,137.0,False,0,168.06,179.12,39,0,0,1,0,0,0,33.0,20.0,15.0,5.0,15.0,10.0,5.0,8.0,9.0,-1.0,-2.0,-4224.25,-8.0,-4009.26,6.0,2.0,-2,25.75,8.0,-6.0,13.0,5.0,8.0,8.0,1.0,7.0,2.0,3.0,-1.0
1,31.0,178.0,29525.52,28.0,148.0,False,1,165.87,176.01,37,1,0,0,0,0,0,30.0,3.0,3.0,0.0,3.0,3.0,0.0,7.0,7.0,0.0,0.0,-2945.92,-15.0,-3984.54,15.0,3.0,-1,15.65,18.0,-15.0,0.0,-3.0,3.0,-4.0,-6.0,2.0,5.0,5.0,0.0
2,31.0,182.0,30820.89,29.0,158.0,False,2,169.35,178.09,34,0,0,0,0,1,0,24.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-3235.06,-26.0,-5813.27,26.0,4.0,-1,14.95,30.0,-26.0,1.0,0.0,1.0,-4.0,-4.0,0.0,-2.0,-2.0,0.0
3,31.0,174.0,30144.62,30.0,152.0,False,3,173.24,181.69,43,0,0,0,0,0,1,22.0,-5.0,-5.0,0.0,-2.0,-2.0,0.0,1.0,1.0,0.0,8.0,-2312.89,-15.0,-4577.58,23.0,-1.0,3,17.76,22.0,-23.0,-6.0,-7.0,1.0,-3.0,-4.0,1.0,0.0,0.0,0.0
4,31.0,179.0,32412.13,35.0,149.0,False,4,181.07,187.83,43,0,0,0,1,0,0,30.0,6.0,4.0,2.0,2.0,0.0,2.0,2.0,0.0,2.0,1.0,-4464.21,-12.0,-4743.03,13.0,3.0,2,22.86,16.0,-13.0,-1.0,-4.0,3.0,0.0,-4.0,4.0,2.0,0.0,2.0


In [9]:
lm = LinearRegression()
lr_model = lm.fit(X1_train, y1_train)
scores = cross_val_score(lm, X1_train, y1_train, scoring='r2', cv=5)
scores.mean()

0.7200482002998536

In [10]:
lr_model.score(X1_test, y1_test)

-0.3065913778547442

In [11]:
rfm = RandomForestRegressor(n_jobs=-1, random_state=20)
rf_model = rfm.fit(X1_train, y1_train)
scores = cross_val_score(rfm, X1_train, y1_train, scoring='r2', cv=5)
scores.mean()

0.6629948201445237

In [12]:
rf_model.score(X1_test, y1_test)

0.34271206291251166

In [16]:
xgbm = XGBRegressor(n_jobs=-1, random_state=20)
xgb_model = xgbm.fit(X1_train, y1_train)
scores = cross_val_score(xgbm, X1_train, y1_train, scoring='r2', cv=5)
scores.mean()

0.6891447167302646

In [17]:
xgbm.score(X1_test, y1_test)

0.10568133288851345

## Random Forest had the highest preliminary test score.

Let's use RandomizedSearchCV to tune the parameters.

Parameters of random grid search
```
random_grid = {
    "n_estimators": range(200, 2000, 100),
    "max_features": ["auto", "sqrt"],
    "max_depth": range(10, 110, 11),
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False]
}

rf = RandomForestRegressor()
rf_random = (RandomizedSearchCV(rf, random_grid, verbose=2, n_iter=50, random_state=42, n_jobs=-1))

rf_random.fit(X1_train, y1_train)
```

Results of random grid search:

```
{'n_estimators': 500,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 43,
 'bootstrap': True}
```

Score: 0.6519058137402494

In [58]:
rf_grid = {
    "n_estimators": [405, 410, 415, 420, 425, 430],
    "max_features": ['auto', 'log2', 70],
    "max_depth": [41, 42, 43, 44, 45],
    "bootstrap": [True]
}
rfm = RandomForestRegressor()

rf_grid = GridSearchCV(rfm, rf_grid, n_jobs=-1, verbose=10, cv=5, random_seed=20)
rf_grid.fit(X1_train, y1_train)

TypeError: __init__() got an unexpected keyword argument 'random_seed'

In [53]:
rf_grid.best_params_

{'bootstrap': True,
 'max_depth': 44,
 'max_features': 'auto',
 'n_estimators': 420}

In [54]:
rf_grid.best_score_

0.656012479002637

In [37]:
# from keras.models import Sequential
# from keras.layers import LSTM


# lstm_model = Sequential()
# model.add(LSTM(4, input_shape=(31, len)